# Renewable Energy Feasibility Orchestrator

---
## Introduction
In this notebook, we will build an **Agentic Workflow** using `semantic-kernel` where multiple AI agents collaborate to produce a **Renewable Energy Feasibility Report**.  

  <div align="center">
<img src="lesson_6.png" alt="Alt text" width="500"/>
</div>

The workflow includes:
- **Orchestrator Agent**: Plans and synthesizes results.
- **Policy Agent**: Collects and summarizes regulatory and policy frameworks.
- **Cost Agent**: Estimates costs and investment requirements.
- **Technology Agent**: Identifies emerging renewable technologies and their readiness.

The orchestrator dynamically assigns tasks to specialized agents and then compiles the results into a **final aggregated feasibility study**.

---

## Step 1: Environment Setup
We start by loading required libraries and environment variables.

In [1]:
import os
import asyncio
from dotenv import load_dotenv

from semantic_kernel.agents import (
    ChatCompletionAgent, StandardMagenticManager, MagenticOrchestration,
)
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.agents.runtime import InProcessRuntime

# Load environment variables
load_dotenv()
api_key = os.getenv("AZURE_OPENAI_KEY")
url = os.getenv("URL")
api_version = "2024-12-01-preview"

## Step 2: Define the Target Industry
We will focus on **renewable energy** instead of aviation.

In [2]:
target_industry = "renewable energy"

## Step 3: Define Azure Chat Completion Service
This connects our agents to Azure OpenAI.

In [3]:
chat_service = AzureChatCompletion(
    deployment_name="none",
    api_key=api_key,
    base_url=url,
    api_version=api_version,
)

## Step 4: Define Agents
Each agent has a **role-specific instruction**.

In [4]:
# Orchestrator
orchestrator = ChatCompletionAgent(
    name="Orchestrator",
    description="Plans, delegates, and synthesizes a renewable energy feasibility report.",
    instructions="",
    service=chat_service,
)

# Policy Agent
policy_agent = ChatCompletionAgent(
    name="PolicyAgent",
    description="Analyzes renewable energy policy and regulations.",
    instructions="You are a policy analyst. Execute only the given instruction.",
    service=chat_service,
)

# Cost Agent
cost_agent = ChatCompletionAgent(
    name="CostAgent",
    description="Estimates renewable energy investment and operational costs.",
    instructions="You are a financial analyst. Execute only the given instruction.",
    service=chat_service,
)

# Technology Agent
tech_agent = ChatCompletionAgent(
    name="TechnologyAgent",
    description="Identifies emerging renewable technologies and evaluates readiness.",
    instructions="You are a technology analyst. Execute only the given instruction.",
    service=chat_service,
)

# List of agents
AGENTS = {
    "Orchestrator": orchestrator,
    "PolicyAgent": policy_agent,
    "CostAgent": cost_agent,
    "TechnologyAgent": tech_agent,
}

available_agents = [a for a in AGENTS.keys() if a != "Orchestrator"]

## Step 5: Define Orchestrator Instructions
The orchestrator dynamically creates a JSON plan.

In [5]:
orchestrator.instructions = (
    f"You are the Orchestrator. Always generate a JSON plan for the "
    f"'{target_industry}' industry.\n"
    f"- The 'title' must explicitly include '{target_industry}'.\n"
    f"- Each step must reference '{target_industry}'.\n"
    f"- You may ONLY assign tasks to these agents: {', '.join(available_agents)}.\n"
    "- You may assign multiple steps to the same agent.\n"
    "After generating the plan, delegate each step to workers by prefixing with '@WorkerName'. "
    "Finally, after all worker replies, include a synthesis step '@Orchestrator' that combines results."
)

## Step 6: Orchestration Setup
We define a callback to track responses.

In [6]:
MAX_ITERATIONS = 6
iteration_count = 0
agents_used = set()
history = []

def track_response(msg: ChatMessageContent, runtime: InProcessRuntime = None) -> None:
    global iteration_count
    iteration_count += 1
    agents_used.add(msg.name)
    history.append(msg)
    print(f"\n--- Iteration {iteration_count}: {msg.name} ---\n{msg.content}")

    if iteration_count >= MAX_ITERATIONS:
        print(f"\n[Manager] Reached maximum iteration limit ({MAX_ITERATIONS}). Stopping orchestration.\n")
        if runtime is not None:
            asyncio.create_task(runtime.stop_when_idle())

manager = StandardMagenticManager(chat_completion_service=chat_service)
orchestration = MagenticOrchestration(
    members=[orchestrator, policy_agent, cost_agent, tech_agent],
    manager=manager,
    agent_response_callback=lambda msg: track_response(msg, runtime=None),
)

## Step 7: Run Orchestration
The workflow has two phases:
1. Generate JSON plan.
2. Execute plan with agents.

In [8]:
async def main():
    global orchestration
    runtime = InProcessRuntime()
    runtime.start()

    orchestration.agent_response_callback = lambda msg: track_response(msg, runtime)

    # Phase 1: Generate JSON plan
    plan_prompt = (
        f"Generate a renewable energy feasibility plan for the target industry: {target_industry}. "
        f"Use only these agents: {', '.join(available_agents)}.\n"
        "Format:\n"
        "{\n"
        f"  \"title\": \"Feasibility plan for the {target_industry} industry\",\n"
        "  \"steps\": [ ... ]\n"
        "}\n"
    )

    plan = ""
    async for msg in orchestrator.invoke(task=plan_prompt, runtime=runtime):
        plan += str(msg)
    print("\n=== Execution Plan ===")
    print(plan)

    # Phase 2: Execute plan
    execution_prompt = (
        plan + "\n\nNow execute the above plan exactly as written. "
        "Use '----- @WorkerName -----' to assign each subtask, "
        "and finish with '@Orchestrator' synthesis."
    )

    orchestration_result = await orchestration.invoke(task=execution_prompt, runtime=runtime)
    final_report = await orchestration_result.get()

    print("\n=== Final Feasibility Report ===\n")
    print(final_report)

    # Summary
    unused_agents = set(available_agents) - agents_used
    print("\n=== Orchestration Summary ===")
    print(f"Total iterations: {iteration_count}")
    print(f"Agents used: {', '.join(agents_used) if agents_used else 'None'}")
    print(f"Agents not used: {', '.join(unused_agents) if unused_agents else 'None'}")
    if iteration_count >= MAX_ITERATIONS:
        print(f"Execution stopped early due to max iteration limit ({MAX_ITERATIONS}).")

    await runtime.stop_when_idle()

await main()


=== Execution Plan ===
{
  "title": "Strategic Plan for Advancing Renewable Energy Adoption",
  "steps": [
    {
      "step": "Analyze current government policies related to renewable energy and propose improvements",
      "agent": "PolicyAgent"
    },
    {
      "step": "Evaluate the cost-benefit analysis for deploying new renewable energy infrastructure",
      "agent": "CostAgent"
    },
    {
      "step": "Research emerging technologies in the renewable energy sector and assess their scalability",
      "agent": "TechnologyAgent"
    },
    {
      "step": "Identify barriers to widespread renewable energy adoption and suggest mitigation strategies",
      "agent": "PolicyAgent"
    },
    {
      "step": "Assess lifecycle costs and financial incentives for renewable energy projects",
      "agent": "CostAgent"
    },
    {
      "step": "Develop a roadmap for integrating advanced renewable energy technologies into existing grids",
      "agent": "TechnologyAgent"
    }
  ]
}

